In [18]:
import sys
sys.executable
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import keras
from pandas import DataFrame
from pandas import read_csv
import math
from numpy import savetxt
from keras import layers
from tensorflow.keras.layers import Input, LSTM, Dense, Reshape, Dropout
from tensorflow.keras.models import Model, Sequential
from scipy.integrate import odeint, RK45
from tensorflow.keras.utils import plot_model
import timeit
tf.keras.backend.set_floatx('float64')

tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [19]:
# Use ODEINT to solve the differential equations defined by the vector field
from scipy.integrate import odeint

def vectorfield(w, t, p):
    """
    Defines the differential equations for the coupled spring-mass system.

    Arguments:
        w :  vector of the state variables:
                  w = [x1,y1,x2,y2]
        t :  time
        p :  vector of the parameters:
                  p = [m1,m2,k1,k2,L1,L2,b1,b2]
    """
    x1, v1, x2, v2, x3, v3 = w
    m, k, kp, d, u1, u2, dist = p

    # Create f = (x1',y1',x2',y2'):
    f = [v1,
         (k * ((-2 * x1) + x2) + kp * (-x1 ** 3 + (x2 - x1) ** 3) + d * (v2-2*v1)) / m + u1,
         v2,
         (k * (x1 - (2 * x2) + x3) + kp * ((x3 - x2) ** 3 - (x2 - x1) ** 3) + d * (v1-2*v2+v3)) / m + u2,
         v3,
         (k * (x2 - x3) + kp * ((x2 - x3) ** 3) + d* (v2-v3)) / m + dist]
    return f

In [20]:
num_rollouts= 1
# Masses:
m = 0.5 
# Spring constants
kp = 63.5
k = 217.0
#damping constant
d= 0.15
# ODE solver parameters
abserr = 1.0e-8
relerr = 1.0e-6

num_data= 100003
interval= 0.001
stoptime = interval*(num_data)
np.random.seed(10)
hist= np.zeros((1,15))

for roll in range(1):
    act1= 3000*(np.random.rand(num_data)-0.5)
    act2= 3000*(np.random.rand(num_data)-0.5)
    dist = 1000*(np.random.rand(num_data)-0.5)
    #Initial states
    w0= np.zeros((1,6))
    #w0 = np.random.randn(1,6) 
    w0= w0.flatten()
    prev = [[w0[0],w0[2],w0[4]]]
    # Pack up the parameters and initial conditions:
    p = [m, k, kp, d, act1[0], act2[0], dist[0]]
    # Call the ODE solver.
    t1= np.array([0,interval])
    wsol1 = odeint(vectorfield, w0, t1, args=(p,),
            atol=abserr, rtol=relerr)
    wsol1 = wsol1.flatten()
    wcurr = np.array([wsol1[6:]])
    w0=wsol1[6:]
    prevv= prev
    prev = [[w0[0],w0[2],w0[4]]]
    # Pack up the parameters and initial conditions:
    p = [m, k, kp, d, act1[1], dist[1], act2[1]]
    # Call the ODE solver.
    t2= np.array([0+interval,interval+interval])
    wsol1 = odeint(vectorfield, w0, t2, args=(p,),
            atol=abserr, rtol=relerr)
    wsol1 = wsol1.flatten()
    wcurr = np.array([wsol1[6:]])
    w0=wsol1[6:]
    curr = np.hstack((prevv, prev, np.array([[act1[1]]]),np.array([[dist[1]]]),np.array([[act2[1]]]),wcurr))
    hist= np.vstack((hist, curr))
   
    lag=2
    for ts in range(num_data-lag):
        prevv = prev
        t = np.array([stoptime * float(ts+lag) / (num_data), stoptime * float(ts + lag + 1) / (num_data)])
        p = [m, k, kp, d, act1[ts+lag],  dist[ts+lag], act2[ts+lag]]
        # Call the ODE solver.
        wsol1 = odeint(vectorfield, w0, t, args=(p,),
            atol=abserr, rtol=relerr)
        wsol1 = wsol1.flatten()
        w0 = wsol1[6:]
        prev = np.array([[wsol1[0],wsol1[2],wsol1[4]]])
        action= [act1[ts+lag],act2[ts+lag]]
        new = np.hstack((prevv,prev,np.array([[act1[ts+lag]]]),np.array([[dist[ts+lag]]]),np.array([[act2[ts+lag]]]),np.array([w0])))
        hist = np.vstack((hist, new))

hist=pd.DataFrame(data=hist,columns =["prev11", "prev22","prev33","prev1","prev2","prev3","u1","dist.","u2","x1", "v1", "x2", "v2", "x3", "v3"])
df=hist[1:][{"u1","dist.","u2","prev11","prev22","prev33","prev1","prev2","prev3","x1", "v1", "x2", "v2", "x3", "v3"}]
#df=hist[1:][{"u1","u2","dist.","x1", "x3"}]
#df=hist[1:]
#df = df[['u1','u2','x1','x3']]
df = df[['u1','dist.','u2',"prev11","prev22","prev33","prev1","prev2","prev3",'x1','v1','x2','v2','x3','v3']]
df

,u1,dist.,u2,prev11,prev22,prev33,prev1,prev2,prev3,x1,v1,x2,v2,x3,v3
1,-1437.744152,-93.426225,1300.434355,0.000000,0.000000,0.000000,0.000407,0.000290,-0.000133,0.000502,-0.624147,0.000823,0.485875,0.000252,1.035069
2,400.944705,-386.471174,135.404101,0.000407,0.000290,-0.000133,0.000502,0.000823,0.000252,0.000078,-0.222763,0.001115,0.099009,0.001355,1.170401
3,746.411648,-219.567159,-423.286406,0.000502,0.000823,0.000252,0.000078,0.001115,0.001355,0.000229,0.523997,0.001104,-0.120460,0.002313,0.746599
4,-4.478963,306.498243,602.361098,0.000078,0.001115,0.001355,0.000229,0.001104,0.002313,0.000751,0.519366,0.001137,0.186810,0.003361,1.348023
5,-825.610063,-339.456666,938.093409,0.000229,0.001104,0.002313,0.000751,0.001137,0.003361,0.000857,-0.306530,0.001155,-0.151091,0.005177,2.284448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,313.512723,126.065577,-658.638362,1.306643,1.820641,1.795204,1.293921,1.841618,1.794379,1.280090,-13.947184,1.862427,20.717079,1.793183,-1.510078
99999,-965.372567,-418.615338,1274.429300,1.293921,1.841618,1.794379,1.280090,1.862427,1.793183,1.265399,-15.429215,1.882770,19.964687,1.792329,-0.196618
100000,-842.838232,-260.476655,187.581300,1.280090,1.862427,1.793183,1.265399,1.882770,1.792329,1.249306,-16.751705,1.902425,19.341175,1.792249,0.038605
100001,-1448.937589,-297.275970,-947.072773,1.265399,1.882770,1.792329,1.249306,1.902425,1.792249,1.231606,-18.640871,1.921424,18.650605,1.791841,-0.852346


In [21]:
NN_in= df[{"u1","u2","dist."}]
NN_in= NN_in[['u1','dist.','u2']].shift(-1)
NN_label= df[{"x1", "x3"}]
NN_label= NN_label[ ["x1", "x3"] ]
NN_previous= df[{"prev11","prev33","prev1","prev3"}]
NN_previous= NN_previous[ ["prev11","prev33"] ]
diff1= df["x1"] - df["prev1"]
diff3= df["x3"] - df["prev3"]
diff21= df["x1"] - df["prev11"]
diff23= df["x3"] - df["prev33"]
df.insert(7, "diff21", diff21)
df.insert(8, "diff23", diff23)
df.insert(9, "diff1", diff1)
df.insert(10, "diff3", diff3)
bothdiff= df[["diff1","diff3"]].shift(0)
bothdiff2= df[["diff21","diff23"]].shift(0)
frames = [NN_in,bothdiff2, NN_label, NN_previous, bothdiff]
df = pd.concat(frames, axis=1)
af=df.dropna()

In [1]:
#max(df[{'diff1'}].values)
plot_cols = ['u1','dist.','u2']
plot_features = df[plot_cols][:]
_ = plot_features.plot(subplots=True)
plot_states = ['x1', 'v1', 'x2', 'v2', 'x3', 'v3']
plot_states = ['x1', 'x3']
plot_features = df[plot_states][:]
_ = plot_features.plot(subplots=True)
plot_out= ['x1','x3']
plot_features = df[plot_out][1:5000]
_ = plot_features.plot(subplots=True)

NameError: name 'df' is not defined

In [23]:
df.std()

u1        865.390832
dist.     288.488131
u2        869.423582
diff21      0.061498
diff23      0.083909
x1          1.321361
x3          2.839704
prev11      1.321350
prev33      2.839693
diff1       0.030758
diff3       0.041959
dtype: float64

In [24]:
df= af[ ["u1","dist.","u2","x1", "x3","diff21","diff23","diff1","diff3"] ][:5000]
#df= af[ ["u1","dist.","u2","x1", "x3","diff1","diff3"] ]

df.to_csv('u1500dampvalidationdatanewage5k_0.001ssim.csv',index=False)
df

,u1,dist.,u2,x1,x3,diff21,diff23,diff1,diff3
1,400.944705,-386.471174,135.404101,0.000502,0.000252,0.000502,0.000252,0.000095,0.000385
2,746.411648,-219.567159,-423.286406,0.000078,0.001355,-0.000329,0.001487,-0.000423,0.001103
3,-4.478963,306.498243,602.361098,0.000229,0.002313,-0.000273,0.002061,0.000151,0.000959
4,-825.610063,-339.456666,938.093409,0.000751,0.003361,0.000672,0.002006,0.000522,0.001047
5,-905.811406,-204.795696,-436.336305,0.000857,0.005177,0.000628,0.002864,0.000106,0.001816
...,...,...,...,...,...,...,...,...,...
4996,-1412.569894,467.179670,-1117.356881,-0.935530,-2.043069,-0.009288,0.042192,-0.005107,0.021582
4997,938.941999,-433.568526,-721.999006,-0.941900,-2.021257,-0.011477,0.043394,-0.006370,0.021811
4998,-563.528814,214.020797,-1218.374725,-0.948429,-2.000263,-0.012899,0.042805,-0.006529,0.020994
4999,-1064.826633,176.976226,225.326393,-0.954671,-1.980139,-0.012771,0.041119,-0.006242,0.020125
